<a href="https://colab.research.google.com/github/semrayarimdunya/AyrikMatematik/blob/main/B%C3%BCy%C3%BCkVeriAnalizi_ipynb_adl%C4%B1_not_defterinin_kopyas%C4%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!update-alternatives --set java /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java
!java -version
!pip install pyspark



Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,307 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,582 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/r

javayı **kurduk** ve
"JAVA HOME" **set** ettik

In [5]:
from pyspark import SparkContext
sc = SparkContext.getOrCreate()
sc


<SparkContext master=local[*] appName=pyspark-shell>

sparkın kurulup kurulmadığını test ettik

In [8]:
from pyspark.sql import SparkSession
from pyspark .ml.feature import VectorAssembler , StringIndexer
from pyspark.ml.classification import DecisionTreeClassifier,GBTClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

spark = SparkSession.builder.appName("Spark ML").getOrCreate()

housepriceDF =spark.read.option("inferSchema","true").option("header", "true").csv("/content/drive/MyDrive/Colab Notebooks/BüyükVeriAnalizi/california_housing_train.csv")
housepriceDF.show(20)

vec = VectorAssembler(inputCols=["longitude", "latitude","housing_median_age",
                                 "total_rooms","total_bedrooms", "population", "households","median_income"],
                      outputCol="features")
housepriceDF = vec.transform(housepriceDF)
housepriceDF.show()

indexer = StringIndexer(inputCol="median_house_value",
                        outputCol ="label")

housepriceDF = indexer.fit(housepriceDF).transform(housepriceDF)

housepriceDF   =housepriceDF.select("features" , "label")
housepriceDF.show(20)

classifier=DecisionTreeClassifier()
#makine öğrenmesi yapılıyor

trainDF ,testDF =housepriceDF.randomSplit([0.80 , 0.20])

model =classifier.fit(trainDF)
print("Makine öğrenmesi tamamlandı")
print(model.toDebugString)


resultDF =model.transform(testDF)
resultDF.show(20)
#model başarısını görmek için

eva =MulticlassClassificationEvaluator()
basari= eva.evaluate(resultDF)
print("Başarı oranımız: ",basari)



+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|
|  -114.47|    34.4|              19.0|     7650.0|        1901.0|    1129.0|     463.0|         1.82|           80100.0|
|  -114.56|   33.69|              17.0|      720.0|         174.0|     333.0|     117.0|       1.6509|           85700.0|
|  -114.57|   33.64|              14.0|     1501.0|         337.0|     515.0|     226.0|       3.1917|           73400.0|
|  -114.57|   33.57|              20.0|     1454.0|         326.0|     624.0|     262.0|        1.925|           65500.0|
|  -114.58|   33.63|    

**sparKSession** ı spark veritabanından import ettik

Daha sonra ML kütüphanesinden **VectorAssembler** ı import ettik.VectorAssembler ile giriş kolonlarını birleştirip tek kolon halinde çıktı verdik vec objesine atayarak yeni bir kolon oluşturduk.

**spark.read** ile dosyayı okuduk ,

**option("inferSchema","true")** ile stringleri int olarak algılamasını sağladık,

**option("header", "true")** ile dosydaki başlıkları dosya verisi olarak algılamasın başlık olarak alsın diye ekledik.

**vec.transform** ile veri dönüşümü yaptık.

**show()** ile dosyayı gösterdik

In [ ]:

import seaborn as sb
import seaborn as sns

kor_data=housepriceDF.drop("longitude","latitude","housing_median_age","total_rooms" "total_bedrooms").toPandas()
sb.pairplot(kor_data)

sb.jointplot(x = 'housing_median_age', y = 'label', data = kor_data)

result_df = kor_data.groupBy("housing_median_age").count().sort("housing_median_age", ascending=False)
result_df.show(10)


NameError: name 'housepriceDF' is not defined

# Yeni Bölüm